In [2]:
from sklearn.cross_validation import StratifiedKFold
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
import numpy as np

try:
    import cPickle as pickle
except:
    import pickle

batch_size = 32
nb_classes = 100
nb_epoch = 10
data_augmentation = True

# input image dimensions
img_rows, img_cols = 32, 32
# the CIFAR10 images are RGB
img_channels = 3

def load_data():
    # load your data using this function
    f = open("../dataset/myfood100-32.pkl", 'rb')

    d = pickle.load(f)
    data = d['trainFeatures']
    labels = d['trainLabels']
    lz = d['labels']
    data = data.reshape(data.shape[0], 3, 32, 32)
    #data = data.transpose(0, 2, 3, 1)

    return data,labels,lz

def create_model():
    # create your model using this function
    model = Sequential()

    model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=(3,32,32)))
    model.add(Activation('relu'))
    model.add(Convolution2D(32, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    return model

def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    
    print "Training.."
    
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
    if not data_augmentation:
        print('Not using data augmentation.')
        #model.fit(X_train, Y_train,
        #          batch_size=batch_size,
        #          nb_epoch=nb_epoch,
        #          validation_data=(X_test, Y_test),
        #          shuffle=True)
        
        model.fit(X_train, Y_train, nb_epoch=nb_epoch, batch_size=batch_size, verbose=0)

        scores = model.evaluate(X_test, Y_test, verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        
    
    else:
        print('Using real-time data augmentation.')

        # this will do preprocessing and realtime data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images

        # compute quantities required for featurewise normalization
        # (std, mean, and principal components if ZCA whitening is applied)
        datagen.fit(X_train)

        # fit the model on the batches generated by datagen.flow()
        model.fit_generator(datagen.flow(X_train, Y_train,
                            batch_size=batch_size),
                            samples_per_epoch=X_train.shape[0],
                            nb_epoch=nb_epoch,
                            validation_data=(X_test, Y_test))
        
        scores = model.evaluate(X_test, Y_test, verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        
    return scores

if __name__ == "__main__":
    n_folds = 2
    cvscores = []
    print "Loading data"
    data, labels, lz = load_data()
    data = data.astype('float32')
    data /= 255
    lz = np.array(lz)
    

    skf = StratifiedKFold(y=lz, n_folds=n_folds, shuffle=True)

    for i, (train, test) in enumerate(skf):
        print ("Running Fold", i+1, "/", n_folds)
        model = None # Clearing the NN.
        model = create_model()
        scores = train_and_evaluate_model(model, data[train], labels[train], data[test], labels[test])
        cvscores.append(scores[1] * 100)
        
    print("Average acc : %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
        


Loading data
('Running Fold', 1, '/', 2)
Training..
Using real-time data augmentation.
Epoch 1/100
10000/10000 [==============================] - 11s - loss: 4.6084 - acc: 0.0089 - val_loss: 4.6052 - val_acc: 0.0116
Epoch 2/100
10000/10000 [==============================] - 11s - loss: 4.6070 - acc: 0.0093 - val_loss: 4.6038 - val_acc: 0.0135
Epoch 3/100
10000/10000 [==============================] - 11s - loss: 4.5663 - acc: 0.0152 - val_loss: 4.4434 - val_acc: 0.0260
Epoch 4/100
10000/10000 [==============================] - 12s - loss: 4.4146 - acc: 0.0243 - val_loss: 4.3398 - val_acc: 0.0280
Epoch 5/100
10000/10000 [==============================] - 11s - loss: 4.3205 - acc: 0.0335 - val_loss: 4.1832 - val_acc: 0.0502
Epoch 6/100
10000/10000 [==============================] - 11s - loss: 4.1953 - acc: 0.0497 - val_loss: 4.1063 - val_acc: 0.0618
Epoch 7/100
10000/10000 [==============================] - 12s - loss: 4.0871 - acc: 0.0604 - val_loss: 4.0195 - val_acc: 0.0765
Epoch 8/10

In [3]:
data_augmentation = False
cvscores = []
for i, (train, test) in enumerate(skf):
        print ("Running Fold", i+1, "/", n_folds)
        model = None # Clearing the NN.
        model = create_model()
        scores = train_and_evaluate_model(model, data[train], labels[train], data[test], labels[test])
        cvscores.append(scores[1] * 100)
        
print("Average acc : %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

('Running Fold', 1, '/', 2)
Training..
Not using data augmentation.
acc: 16.59%
('Running Fold', 2, '/', 2)
Training..
Not using data augmentation.
acc: 16.54%
Average acc : 16.56% (+/- 0.03%)


NameError: name 'model' is not defined